In [1]:
from sklearn.datasets import fetch_20newsgroups
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
from nltk.stem import PorterStemmer
import numpy as np
import nltk
import pandas as pd
import os
nltk.download('names')
from sklearn.feature_extraction.text import TfidfVectorizer
ps = PorterStemmer()
all_names = names.words()
WNL = WordNetLemmatizer()

path_train= "/home/antonis/Downloads/20news-bydate/20news-bydate-train"



def gather(path):
    df = pd.DataFrame()
    for file in os.listdir(path):
        tag = file
        for doc in os.listdir(path+'/'+file):
            docpath = path+'/'+file+'/'+doc
            f = open(docpath, "r",encoding='cp1252')
            content = f.read()
            temp = pd.DataFrame(
                {
                    'content':content,
                    'tag':tag
                },index=[0]
            )
            df = pd.concat([df, temp])



    df.content =df.content.replace(to_replace='From:(.*\n)', value='', regex=True) ##remove from to email
    df.content =df.content.replace(to_replace='lines:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='Subject:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]', value=' ', regex=True) #remove punctuation except
    df.content =df.content.replace(to_replace='-', value=' ', regex=True)
    df.content =df.content.replace(to_replace='\s+', value=' ', regex=True)    #remove new line
    df.content =df.content.replace(to_replace='  ', value='', regex=True)                #remove double white space
    df.content =df.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

    df['content']=[entry.lower() for entry in df['content']]
    return df

df_news_train = gather(path_train)



def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                words = ps.stem(words)
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1
    return(list(cleaned.values()))

x_train = clean(df_news_train['content'])


tf = TfidfVectorizer(stop_words='english', max_features=8000,use_idf=True)
tfidf = tf.fit_transform(x_train)




[nltk_data] Downloading package names to /home/antonis/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [1]:
from sklearn.feature_extraction.text import CountVectorizer

vocab = tf.vocabulary_

countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english',vocabulary=vocab)

count_wm = countvectorizer.fit_transform(x_train)


NameError: name 'tf' is not defined

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
from numpy import asarray
from numpy import savetxt


#using Tfidftransformer and count vectorizer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(count_wm)


# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=countvectorizer.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

# count matrix 
count_vector=countvectorizer.transform(x_train) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)


feature_names = countvectorizer.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[10342] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=['tfidf'],ascending=False)

#using Tfidfvectorizer

# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf[0] 

# place tf-idf values in a pandas data frame
df_tfidf = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tf.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)


print(df_tfidf)

def get_key(val):
    for key, value in tf.vocabulary_.items():
         if val == value:
             return key
 
    return "key doesn't exist"




         tfidf
aa         0.0
aaa        0.0
aaron      0.0
ab         0.0
abandon    0.0
...        ...
zubov      0.0
zurich     0.0
zv         0.0
zx         0.0
zz         0.0

[8000 rows x 1 columns]


In [6]:
import json

f = open('/home/antonis/PycharmProjects/soup/news/Huffpost.json')
data = json.load(f)

documents = []

for i in data:
    documents.append(data[i])


In [29]:
#instantiate CountVectorizer() 
cv=CountVectorizer() 
 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(documents)

print(word_count_vector.shape)

(55, 4773)


TypeError: 'Series' objects are mutable, thus they cannot be hashed